In [1]:
import os
import pandas as pd
import numpy as np
import tkinter as tk
from xlsx2html import xlsx2html
import xlsxwriter
import tkinter.filedialog as fd
import time
from openpyxl.workbook import Workbook
from openpyxl.reader.excel import load_workbook
import time
import datetime

In [2]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a Base file')
root.destroy()
df = pd.read_excel(file, skiprows=9, keep_default_na=False)

In [3]:
df['Total Weight shipped (kg)'].dtype
df['Total Weight shipped (kg)'] = pd.to_numeric(df['Total Weight shipped (kg)'], errors='coerce')
df['Total Weight shipped (kg)'] = df['Total Weight shipped (kg)'].fillna(0)
df2=df

df2['Has Shipped Weight'] = df2['Total Weight shipped (kg)'].apply(
    lambda x: 'Yes' if pd.notna(x) and x != 0 else 'No'
)
df2 = df2[df2['Sub Sector'] != 'Personal Health Care']
df2= df2[df2['Has Shipped Weight'] == 'Yes']
df2['Consumer Packagining'] = df2['BOM Packing Level'].apply(lambda x: 'Yes' if x == 'Consumer Packaging' else 'No')

df2= df2[df2['Consumer Packagining'] == 'Yes']

df2['B-to-B / Minor Brands'] = df2['Brand'].apply(lambda x: 'No' if 'Minor Brands' in str(x) else 'Yes')

df2= df2[df2['B-to-B / Minor Brands'] == 'Yes']

output_file = 'Base_file.xlsx'
df2.to_excel(output_file, index=False)

### lcm Type

In [4]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a LCM Master file')
root.destroy()
LCM_Master = pd.read_excel(file, keep_default_na=False)

In [5]:
df3=df

In [6]:
df3 = df[['Legacy Environmental Class','Total Weight shipped (kg)']]

LCM_with_data = df3.groupby('Legacy Environmental Class')['Total Weight shipped (kg)'].sum().reset_index()
LCM_Type=pd.merge(LCM_with_data,LCM_Master[['Legacy Environmental Class','Classification']], left_on='Legacy Environmental Class', right_on='Legacy Environmental Class', how='left')

LCM_Type['Comment']= ' '
output_file_with_data = 'LCM_Type.xlsx'

LCM_Type.to_excel(output_file_with_data, index=False)

In [7]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a LCM Type file')
root.destroy()
LCM = pd.read_excel(file, keep_default_na=False)


LCM_Type_new=pd.merge(df2,LCM[['Legacy Environmental Class','Classification']], left_on='Legacy Environmental Class', right_on='Legacy Environmental Class', how='left')

LCM_Type_new = LCM_Type_new.rename(columns={'Classification':'Initial MCP Classification'})


output_file_with_data = 'LCM_Type_new.xlsx'
LCM_Type_new.to_excel(output_file_with_data, index=False)

In [8]:
dff = LCM_Type_new

# 1) Pivot
Composite_Mat_Check = (
    dff.groupby(['Packaging Part Name', 'Initial MCP Classification'])['MCP Percentage']
      .sum().reset_index()
      .pivot(index='Packaging Part Name', columns='Initial MCP Classification', values='MCP Percentage')
      .reset_index()
    .fillna(0)
)

# 2) Detect columns (keep each Glass variant separate)
all_cols = [c for c in Composite_Mat_Check.columns if isinstance(c, str)]
glass_cols  = [c for c in all_cols if c.lower().startswith('glass')]   # e.g., Glass Other, Glass Colourless, Glass Amber, Glass
other_cols  = [c for c in ['Plastic','Paper', 'PET', 'NA'] if c in Composite_Mat_Check.columns]
present_materials = glass_cols + other_cols  # only what exists; skip missing

# 3) Totals
Composite_Mat_Check['Total'] = Composite_Mat_Check[present_materials].sum(axis=1)
Composite_Mat_Check['Total_check'] = Composite_Mat_Check['Total'] / 100.0

# 4) Normalize per material (create *_check columns only for present materials)
for m in present_materials:
    Composite_Mat_Check[f'{m}_check'] = (
        Composite_Mat_Check[m] / Composite_Mat_Check['Total_check'].replace(0, np.nan)
    )

# 5) Clean NaNs from divide-by-zero
check_cols = [f'{m}_check' for m in present_materials]
Composite_Mat_Check[check_cols] = Composite_Mat_Check[check_cols].fillna(0)

# 6) Drop originals & helper cols (keep only normalized material columns, plus Packaging Part Name)
Composite_Mat_Check.drop(columns=present_materials + ['Total_check'], errors='ignore', inplace=True)

# 7) Rename *_check back to original names
Composite_Mat_Check.rename(columns={f'{m}_check': m for m in present_materials}, inplace=True)

# 8) Recompute Total/Max on the normalized columns
Composite_Mat_Check['Total'] = Composite_Mat_Check[present_materials].sum(axis=1).round()
Composite_Mat_Check['Max']   = Composite_Mat_Check[present_materials].max(axis=1)

# 9) Priority for tie-breaks: put Glass variants first (in a sensible order), then others
glass_priority_order = ['Glass Colourless', 'Glass Amber', 'Glass Other', 'Glass']
priority_cols = [c for c in glass_priority_order if c in Composite_Mat_Check.columns] + \
                [c for c in ['Plastic','Paper', 'PET', 'NA'] if c in Composite_Mat_Check.columns]

# Final classification prefers earlier columns in priority_cols on ties
Composite_Mat_Check['Final Classification'] = Composite_Mat_Check[priority_cols].idxmax(axis=1)

# 10) Save
output_file_with_data = 'Composite_Mat_Check.xlsx'
Composite_Mat_Check.to_excel(output_file_with_data, index=False)

C:\Users\Kamlesh\AppData\Local\Temp\ipykernel_2336\3176179216.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(0)


In [9]:

# dff = LCM_Type_new

# Composite_Mat_Check = dff.groupby(
#     ['Packaging Part Name', 'Initial MCP Classification']
# )['MCP Percentage'].sum().reset_index()

# Composite_Mat_Check = Composite_Mat_Check.pivot(
#     index='Packaging Part Name',
#     columns='Initial MCP Classification',
#     values='MCP Percentage'
# ).reset_index()

# Composite_Mat_Check.fillna(0, inplace=True)

# # --- ensure all expected material columns exist (add 0 if missing)
# for m in ['Glass', 'NA', 'Paper', 'PET', 'Plastic']:
#     if m not in Composite_Mat_Check.columns:
#         Composite_Mat_Check[m] = 0

# Composite_Mat_Check['Total'] = Composite_Mat_Check.sum(axis=1, numeric_only=True)

# # Composite_Mat_Check = Composite_Mat_Check.append(total_row, ignore_index=True)  # (left as-is in your code but commented)

# # Use safe denominator (avoid divide-by-zero)
# den = Composite_Mat_Check['Total'].replace(0, np.nan)
# Composite_Mat_Check['Total_check'] = Composite_Mat_Check['Total'] / 100

# # Use bracket access (works even if a column was just added above)
# Composite_Mat_Check['Glass_check']   = Composite_Mat_Check['Glass']   / (Composite_Mat_Check['Total_check'].replace(0, np.nan))
# Composite_Mat_Check['NA_check']      = Composite_Mat_Check['NA']      / (Composite_Mat_Check['Total_check'].replace(0, np.nan))
# Composite_Mat_Check['Paper_check']   = Composite_Mat_Check['Paper']   / (Composite_Mat_Check['Total_check'].replace(0, np.nan))
# Composite_Mat_Check['PET_check']     = Composite_Mat_Check['PET']     / (Composite_Mat_Check['Total_check'].replace(0, np.nan))
# Composite_Mat_Check['Plastic_check'] = Composite_Mat_Check['Plastic'] / (Composite_Mat_Check['Total_check'].replace(0, np.nan))

# # Any divisions that hit NaN (because Total was 0) become 0
# Composite_Mat_Check[['Glass_check','NA_check','Paper_check','PET_check','Plastic_check']] = \
#     Composite_Mat_Check[['Glass_check','NA_check','Paper_check','PET_check','Plastic_check']].fillna(0)

# # Drop original material columns & helpers (ignore if not present)
# Composite_Mat_Check = Composite_Mat_Check.drop(
#     columns=['Glass','NA','PET','Paper','Plastic','Total','Total_check'],
#     errors='ignore'
# )

# # Rename _check back to original names (as in your code)
# Composite_Mat_Check = Composite_Mat_Check.rename(columns={
#     'Glass_check':'Glass',
#     'NA_check':'NA',
#     'Paper_check':'Paper',
#     'PET_check':'PET',
#     'Plastic_check':'Plastic'
# })

# # Recompute Total & Max with your original approach
# Composite_Mat_Check['Total'] = Composite_Mat_Check.sum(axis=1, numeric_only=True)
# Composite_Mat_Check['Total'] = Composite_Mat_Check['Total'].round()

# Composite_Mat_Check['Max'] = Composite_Mat_Check.iloc[:, 1:-1].max(axis=1)
# Composite_Mat_Check['Final Classification'] = Composite_Mat_Check.iloc[:, 1:-2].idxmax(axis=1)

# output_file_with_data = 'Composite_Mat_Check.xlsx'
# Composite_Mat_Check.to_excel(output_file_with_data, index=False)

In [10]:

root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a Composite_Mat_Check file')
root.destroy()
dff3 = pd.read_excel(file, keep_default_na=False)

df6 = LCM_Type_new

df6['Packaging Part Name']  = pd.to_numeric(df6['Packaging Part Name'],  errors='coerce').astype('Int64')
dff3['Packaging Part Name']  = pd.to_numeric(dff3['Packaging Part Name'],  errors='coerce').astype('Int64')

LCM_Sus_new=pd.merge(df6,dff3[['Packaging Part Name','Final Classification']], left_on='Packaging Part Name', right_on='Packaging Part Name', how='left')

output_file_with_data = 'LCM_Sus_new.xlsx'
LCM_Sus_new.to_excel(output_file_with_data, index=False)


In [11]:
LCM_Sus_new['New_Classification'] = LCM_Sus_new.apply(
    lambda row: (
        "Pet" if row['Final Classification'] == "PET" and row['Packaging Component Type'] in ["BOTTLE", "CANISTER"]
        else "Plastic" if row['Final Classification'] == "PET"
        else row['Final Classification']
    ),
    axis=1
)

In [12]:
output_file_with_data = 'LCM_Sus_new.xlsx'
LCM_Sus_new.to_excel(output_file_with_data, index=False)

### PCM Master

In [13]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a PCM Master file')
root.destroy()
PCM_Master = pd.read_excel(file, keep_default_na=False)

In [14]:
PCM_Master.columns

Index(['Packaging Component Type', 'Classification'], dtype='object')

In [15]:
df4=df2

df4 = df4[['Packaging Component Type', 'Total Weight shipped (kg)']]

df4['Packaging Component Type'].replace('', 'need more data', inplace=True)

# df_with_data = df3[df3['Packaging Component Type'] != 'need more data']
# df_need_more_data = df3[df3['Packaging Component Type'] == 'need more data']

Packaging_Component_Type_with_data = df4.groupby('Packaging Component Type')['Total Weight shipped (kg)'].sum().reset_index()
# Exception_File = df_need_more_data.groupby('Packaging Component Type')['Total Weight shipped (kg)'].sum().reset_index()

C:\Users\Kamlesh\AppData\Local\Temp\ipykernel_2336\1085508957.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df4['Packaging Component Type'].replace('', 'need more data', inplace=True)
C:\Users\Kamlesh\AppData\Local\Temp\ipykernel_2336\1085508957.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Packaging Component Type'].replace('

In [16]:
import unicodedata
left  = Packaging_Component_Type_with_data.copy()
right = PCM_Master[['Packaging Component Type', 'Classification']].copy()

def clean_key(s: pd.Series) -> pd.Series:
    s = s.astype(str)
    s = s.str.replace('\u00A0', ' ', regex=False)        # non-breaking space → normal space
    s = s.map(lambda x: unicodedata.normalize('NFKC', x))# normalize unicode (full-width, accents, etc.)
    s = s.str.strip()                                    # trim
    s = s.str.replace(r'\s+', ' ', regex=True)           # collapse multiple spaces
    s = s.str.replace('[–—‐-]', '-', regex=True)         # unify hyphen variants
    s = s.str.upper()                                    # case-insensitive compare
    return s

# Build normalized keys
left['_key']  = clean_key(left['Packaging Component Type'])
right['_key'] = clean_key(right['Packaging Component Type'])

# If right has duplicates per key, keep first (avoid row explosions)
right_dedup = right.sort_values('Classification').drop_duplicates('_key', keep='first')

# Merge on the normalized key
merged = left.merge(
    right_dedup[['_key', 'Classification']],
    on='_key',
    how='left'
)

# # If you need the Excel logic "if New_Classification=='NA' then 'NA' else the looked up":
# if 'New_Classification' in merged.columns:
#     merged['Classification'] = np.where(
#         merged['New_Classification'].astype(str).str.upper().eq('NA'),
#         'NA',
#         merged['Classification']
#     )

# Clean-up
Packaging_Type = merged.drop(columns=['_key'])

In [17]:

output_file_with_data = 'Packaging_Type.xlsx'
# output_file_need_more_data = 'Exception_File.xlsx'

Packaging_Type.to_excel(output_file_with_data, index=False)

In [18]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a PCM Type file')
root.destroy()
PCM_Type = pd.read_excel(file, keep_default_na=False)



In [19]:
Packaging_Sus=pd.merge(LCM_Sus_new,PCM_Type[['Packaging Component Type','Classification']], left_on='Packaging Component Type', right_on='Packaging Component Type', how='left')


In [20]:
Packaging_Sus['Classification'] = np.where(
    Packaging_Sus['New_Classification'].astype(str).str.upper().eq('NA'),
    'NA',
    Packaging_Sus['Classification']
)


In [21]:

# Packaging_Sus=pd.merge(LCM_Sus_new,PCM_Type[['Packaging Component Type','Classification']], left_on='Packaging Component Type', right_on='Packaging Component Type', how='left')


Packaging_Sus = Packaging_Sus.rename(columns={'Classification':'Final Material Classification'})



In [22]:
Packaging_Sus['Final Material Classification_Copy'] = Packaging_Sus['Final Material Classification']
Packaging_Sus['Final Material Classification'] = Packaging_Sus['Final Material Classification'].replace({'NA': 'Container'})

In [23]:
output_file_with_data = 'Packaging_Sus.xlsx'
Packaging_Sus.to_excel(output_file_with_data, index=False)

### Source Master

In [133]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a Source Master')
root.destroy()
Source_Master = pd.read_excel(file, keep_default_na=False)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [25]:
right = (Source_Master[['Finished Product Part Name','Source']]
         .dropna(subset=['Finished Product Part Name'])
         .drop_duplicates(subset=['Finished Product Part Name'], keep='first'))

# 2) Many-to-one merge (will error if still not many-to-one)
final_file = Packaging_Sus.merge(
    right,
    on='Finished Product Part Name',
    how='left',
    validate='many_to_one'
)

In [26]:
final_file['Source_Copy'] = final_file['Source']
final_file['Source'] = final_file['Source'].replace({'NA': 'JP-Sourced'})

In [27]:
output_file_with_data = 'final_file.xlsx'

final_file.to_excel(output_file_with_data, index=False)

### Mapping Sheet

In [28]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a Mapping Sheet file')
root.destroy()
Mapping_sheet = pd.read_excel(file, skiprows=1,keep_default_na=False)

In [29]:
df8=final_file

In [30]:
# diagnose
Mapping_sheet['Category'] = Mapping_sheet['Category'].astype('string').str.strip()
df8['Category'] = df8['Category'].astype('string').str.strip()

dupes = Mapping_sheet['Category'][Mapping_sheet.duplicated('Category', keep=False)]
# print(dupes.value_counts().head(10))  # optional: see worst offenders

# dedupe right side
map_dedup = (Mapping_sheet[['Category','Business Classification']]
             .dropna(subset=['Category'])
             .drop_duplicates(subset=['Category'], keep='first'))

reporting_category = df8.merge(
    map_dedup,
    on='Category',
    how='left',
    validate='many_to_one'     # will raise if still duplicated
)


In [31]:
reporting_category['Final Classification'] = reporting_category['Final Classification'].replace({'PET': 'Plastic'})

In [32]:
output_file_with_data = 'reporting_category.xlsx'

reporting_category.to_excel(output_file_with_data, index=False)

### Final Base File

In [33]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a Reporting unit cost file')
root.destroy()
unit_cost = pd.read_excel(file,keep_default_na=False)

In [34]:
unit_cost

,Packing,"Glass Bottles, Colorless","Glass Bottles, Amber","Glass Bottles, Other Colors",PET Bottle,Paper Packaging,Plastic Packaging
0,"Unit Cost,\/kg",11,13.9,20.2,8.8,22.0,63.0


In [35]:
root = tk.Tk()
file = fd.askopenfilename(parent=root, title='Choose a Coefficient for Ordinary Calculation file')
root.destroy()
odi_cal = pd.read_excel(file,keep_default_na=False)

In [36]:
reporting=reporting_category

### Rate per KG

In [37]:
def get_cost(final_classification):
    if final_classification == 'Plastic':
        return unit_cost['Plastic Packaging'].values[0]
    elif final_classification == 'Paper':
        return unit_cost['Paper Packaging'].values[0]
    elif final_classification == 'Glass Colorless':
        return unit_cost['Glass Bottles, Colorless'].values[0]
    elif final_classification == 'Glass Other':
        return unit_cost['Glass Bottles, Other Colors'].values[0]
    elif final_classification == 'Glass Amber':
        return unit_cost['Glass Bottles, Amber'].values[0]
    else:
        return 0  # Default value if needed

# Apply the function to the 'Final Classification' column
reporting['Rate per KG'] = reporting['Final Classification'].apply(get_cost)

In [38]:
output_file_with_data = 'reporting.xlsx'

reporting.to_excel(output_file_with_data, index=False)

In [39]:
reporting = pd.merge(
    reporting, 
    odi_cal[['Business Classification', 'Container/Wrapping']],  # Include other necessary columns
    left_on='Business Classification', 
    right_on='Business Classification', 
    how='left'
)

# reporting['Container/Wrapping'] = reporting['Container/Wrapping'].fillna('Container')

### Conatiner

In [40]:
reporting_Con = reporting

In [45]:
reporting_Con['Final Material Classification'].unique()

array(['Container', 'Wrapping', nan], dtype=object)

In [46]:
reporting_Con= reporting_Con[reporting_Con['Final Material Classification'] == 'Container']

In [41]:
reporting_Con['Final Material Classification'].unique()

array(['Container', 'Wrapping', nan], dtype=object)

In [42]:
reporting_Con = pd.DataFrame(reporting_Con)

# Safely handle NaN or missing Source values
reporting_Con['Concatenated Classification'] = reporting_Con.apply(
    lambda row: (
        f"{row['Business Classification']}{row['Final Material Classification']}{row['Final Classification']}{row['Source']}"
        if pd.notna(row['Source']) and "JP-Sourced" in str(row['Source'])
        else f"{row['Business Classification']}{row['Final Material Classification']}{row['Final Classification']}{row['Source']}"
    ),
    axis=1
)

In [43]:
# output_file_with_data = 'reporting_Con.xlsx'
# reporting_Con.to_excel(output_file_with_data, index=False)

In [44]:
odi_cal = pd.DataFrame(odi_cal)

In [45]:
columns_to_unpivot = [
'Glass Colorless Use', 'Glass Colorless Manufacture', 'Glass Amber Use',
'Glass Amber Manufacture', 'Glass Other Use', 'Glass Other Manufacture',
'PET Bottle Use', 'PET Bottle Manufacture', 'Paper Packaging Use',
'Paper Packaging Manufacture', 'Plastic Packaging Use',
'Plastic Packaging Manufacture'
]

In [46]:
unpivoted_df = pd.melt(odi_cal, 
                   id_vars=['Business Classification', 'Container/Wrapping'], 
                   value_vars=columns_to_unpivot,
                   var_name='Material Type', 
                   value_name='Quantity')

In [47]:
unpivoted_df[['Material', 'Type']] = unpivoted_df['Material Type'].str.extract(r'(.+)\s(Use|Manufacture)')

unpivoted_df.drop(columns=['Material Type'], inplace=True)

unpivoted_df['Type'] = unpivoted_df['Type'].replace({'Use': 'JP-Sourced', 'Manufacture': 'Imported'})

In [48]:
unpivoted_df['Material'] = unpivoted_df['Material'].replace({'Plastic Packaging': 'Plastic','Paper Packaging':'Paper'})
unpivoted_df['bsm'] = unpivoted_df.apply(lambda row: f"{row['Business Classification']}{row['Container/Wrapping']}{row['Material']}", axis=1)
unpivoted_df['bst'] = unpivoted_df.apply(lambda row: f"{row['Business Classification']}{row['Container/Wrapping']}{row['Material']}{row['Type']}", axis=1)
unpivoted_df['bst_1'] = unpivoted_df.apply(lambda row: f"{row['Container/Wrapping']}{row['Material']}{row['Type']}", axis=1)

In [49]:
unpivoted_df['Quantity'] = pd.to_numeric(unpivoted_df['Quantity'], errors='coerce')
unpivoted_df['Quantity'] = unpivoted_df['Quantity'].fillna(0)

In [50]:
output_file_with_data = 'unpivoted_df.xlsx'
unpivoted_df.to_excel(output_file_with_data, index=False)

In [51]:
unpivoted_df.columns

Index(['Business Classification', 'Container/Wrapping', 'Quantity', 'Material',
       'Type', 'bsm', 'bst', 'bst_1'],
      dtype='object')

In [52]:
reporting_1 = reporting_Con

In [53]:
reporting_1 = pd.merge(reporting_1, unpivoted_df[['bst','Quantity','Material']],left_on='Concatenated Classification', right_on='bst', how='left')

In [54]:
reporting_1.rename(columns={'Quantity': 'Coefficient (Normal)'}, inplace=True)

In [55]:
reporting_1.columns

Index(['Item GTIN (EAN UPC Bar Code) ', 'Finished Product Part Name',
       'Finished Product Part Revision', 'Finished Product Part Title',
       'Sector', 'Sub Sector', 'Category', 'Brand', 'Primary Organization',
       'Secondary Organization', 'FPP Originator',
       'Number of COP per CUP (Items per Case)', 'Marketing Size',
       'Marketing Size UOM ', 'Statistical Unit Factor ', 'Base Unit Quantity',
       'Base Unit of Measure UOM ', 'Stat Unit Quantity', 'Consumer Items',
       'Region', 'MDO', 'Market of Sale', 'BOM Source', 'Reported Function',
       'BOM Packing Level', 'Quantity per CUP (Case)',
       'Quantity per CUP (Case) UOM', 'BOM Hierarchy (F/N)',
       'Packaging Under Product', 'Direct Parent Part Type',
       'Direct Parent Part Name', 'Direct Parent Part Revision',
       'Direct Parent Part Title', 'Packaging Part Type',
       'Packaging Part Name', 'Packaging Part Revision',
       'Packaging Part Title', 'Packaging Material Type',
       'Packagin

In [56]:
# output_file_with_data = 'reporting_1.xlsx'
# reporting_1.to_excel(output_file_with_data, index=False)

### Wrapper

In [57]:
reporting_Wrap = reporting

In [58]:
reporting_Wrap['Final Material Classification'].unique()

array(['Container', 'Wrapping', nan], dtype=object)

In [59]:
reporting_Wrap= reporting_Wrap[reporting_Wrap['Final Material Classification'] == 'Wrapping']

In [60]:
reporting_Wrap['Final Material Classification'].unique()

array(['Wrapping'], dtype=object)

In [61]:
reporting_Wrap = pd.DataFrame(reporting_Wrap)

# Safely handle NaN or missing Source values
reporting_Wrap['Concatenated Classification'] = reporting_Wrap.apply(
    lambda row: (
        f"{row['Final Material Classification']}{row['New_Classification']}{row['Source']}"
        if pd.notna(row['Source']) and "JP-Sourced" in str(row['Source'])
        else f"{row['Final Material Classification']}{row['New_Classification']}{row['Source']}"
    ),
    axis=1
)

In [62]:
reporting_Wrap = pd.merge(reporting_Wrap, unpivoted_df[['bst_1','Quantity','Material']],left_on='Concatenated Classification', right_on='bst_1', how='left')

In [63]:
material_avg = reporting_Wrap.loc[reporting_Wrap["Quantity"] != 0].groupby("Material")["Quantity"].mean().to_dict()

In [64]:
material_avg

{'Plastic': 0.51923}

In [65]:
reporting_Wrap["Quantity"] = reporting_Wrap.apply(
    lambda row: material_avg.get(row["Material"], 0) if row["Quantity"] == 0 else row["Quantity"],
    axis=1
)

In [66]:
reporting_Wrap.rename(columns={'Quantity': 'Coefficient (Normal)'}, inplace=True)

In [67]:
output_file_with_data = 'reporting_Wrap.xlsx'
reporting_Wrap.to_excel(output_file_with_data, index=False)

In [68]:
reporting_new = pd.concat([reporting_1, reporting_Wrap], ignore_index=True)

In [69]:
output_file_with_data = 'reporting_new.xlsx'
reporting_new.to_excel(output_file_with_data, index=False)

#### Container + Wrapper for JP

In [70]:
reporting_JP= reporting_new

In [71]:
reporting_JP= reporting_JP[reporting_JP['Source'] == 'JP-Sourced']

In [72]:
output_file_with_data = 'reporting_JP.xlsx'
reporting_JP.to_excel(output_file_with_data, index=False)

### Container for Importer

In [73]:
reporting_Imp= reporting_new

In [74]:
reporting_Imp= reporting_Imp[reporting_Imp['Source'] == 'Imported']

In [75]:
reporting_Imp = reporting_Imp.drop(columns=['Coefficient (Normal)', 'Material', 'bst_1'])

In [76]:
reporting_Imp_Cons = reporting_Imp

In [77]:
reporting_Imp_Cons= reporting_Imp_Cons[reporting_Imp_Cons['Final Material Classification'] == 'Container']

In [78]:
reporting_Imp_Cons = pd.merge(reporting_Imp_Cons, unpivoted_df[['bst','Quantity','Material']],left_on='Concatenated Classification', right_on='bst', how='left')

In [79]:
reporting_Imp_Cons.rename(columns={'Quantity': 'Coefficient (Imported)','bst_x':'bst'}, inplace=True)

In [80]:
reporting_Imp_Cons['Coefficient (Imported)'] = reporting_Imp_Cons['Coefficient (Imported)'].fillna(0)

In [81]:
reporting_Imp_Cons['Coefficient (Imported)'].value_counts()

Coefficient (Imported)
0.02614    528
0.00045     22
0.00000      3
Name: count, dtype: int64

In [82]:
reporting_Imp_Cons_jp  =  reporting_Imp_Cons

In [83]:
reporting_Imp_Cons_jp["bst"] = reporting_Imp_Cons_jp["bst"].str.replace(
    "Imported", "JP-Sourced", regex=False
)

In [84]:
# output_file_with_data = 'reporting_Imp_Cons_jp.xlsx'
# reporting_Imp_Cons_jp.to_excel(output_file_with_data, index=False)

In [85]:
reporting_Imp_Cons_jp = pd.merge(reporting_Imp_Cons_jp, unpivoted_df[['bst','Quantity','Material']],left_on='bst', right_on='bst', how='left')

In [86]:
reporting_Imp_Cons_jp.rename(columns={'Quantity': 'Coefficient (Normal)','bst_x':'bst','Material_x':'Material'}, inplace=True)

In [87]:
reporting_Imp_Cons_jp['Coefficient (Normal)'] = reporting_Imp_Cons_jp['Coefficient (Normal)'].fillna(0)

In [88]:
reporting_Imp_Cons_jp['Coefficient (Normal)'].value_counts()

Coefficient (Normal)
0.62631    528
0.02896     22
0.00000      3
Name: count, dtype: int64

In [89]:
# output_file_with_data = 'reporting_Imp_Cons_jp.xlsx'
# reporting_Imp_Cons_jp.to_excel(output_file_with_data, index=False)

### Container for Wrapper

In [90]:
reporting_Imp_wrap= reporting_new

In [91]:
reporting_Imp_wrap= reporting_Imp_wrap[reporting_Imp_wrap['Source'] == 'Imported']

In [92]:
reporting_Imp_wrap = reporting_Imp_wrap.drop(columns=['Coefficient (Normal)', 'Material'])

In [93]:
reporting_Imp_wrap= reporting_Imp_wrap[reporting_Imp_wrap['Final Material Classification'] == 'Wrapping']

In [94]:
reporting_Imp_wrap = pd.merge(reporting_Imp_wrap, unpivoted_df[['bst_1','Quantity','Material']],left_on='bst_1', right_on='bst_1', how='left')

In [95]:
reporting_Imp_wrap.rename(columns={'Quantity': 'Coefficient (Imported)'}, inplace=True)

In [96]:
reporting_Imp_wrap_jp =reporting_Imp_wrap

In [97]:
reporting_Imp_wrap_jp['bst_copy'] = reporting_Imp_wrap_jp['bst_1']
# final_file['Source'] = final_file['Source'].replace({'NA': 'JP-Sourced'})

In [98]:
reporting_Imp_wrap_jp['bst_1'].unique()

array([nan, 'WrappingPlasticImported'], dtype=object)

In [99]:
reporting_Imp_wrap_jp["bst_1"] = reporting_Imp_wrap_jp["bst_1"].str.replace(
    "Imported", "JP-Sourced", regex=False
)

In [100]:
output_file_with_data = 'reporting_Imp_wrap_jp.xlsx'
reporting_Imp_wrap_jp.to_excel(output_file_with_data, index=False)

In [101]:
reporting_Imp_wrap_jp = pd.merge(reporting_Imp_wrap_jp, unpivoted_df[['bst_1','Quantity','Material']],left_on='bst_1', right_on='bst_1', how='left')

In [102]:
reporting_Imp_wrap_jp.rename(columns={'Quantity': 'Coefficient (Normal)','bst_x':'bst','Material_x':'Material'}, inplace=True)

In [103]:
reporting_Imp_wrap_jp['Coefficient (Normal)'] = reporting_Imp_wrap_jp['Coefficient (Normal)'].fillna(0)

In [104]:
reporting_Imp_wrap_jp['Coefficient (Normal)'].value_counts()

Coefficient (Normal)
0.00000    50
0.51923    50
Name: count, dtype: int64

In [105]:
new_reporting = pd.concat([reporting_Imp_Cons_jp, reporting_Imp_wrap_jp], ignore_index=True)

In [106]:
output_file_with_data = 'new_reporting.xlsx'
new_reporting.to_excel(output_file_with_data, index=False)

In [107]:
upload_base_file = pd.concat([reporting_JP, new_reporting], ignore_index=True)

In [108]:
upload_base_file.rename(columns={'Total Weight shipped (kg)':'CY Weight'}, inplace=True)

In [109]:
upload_base_file['Coefficient (Imported)'] = upload_base_file['Coefficient (Imported)'].fillna(0)
upload_base_file['Coefficient (Normal)'] = upload_base_file['Coefficient (Normal)'].fillna(0)


In [110]:
upload_base_file['Recycling Fees (Normal)'] = upload_base_file['CY Weight'] * upload_base_file['Rate per KG'] * upload_base_file['Coefficient (Normal)']

upload_base_file['Recycling Fees (Imported)'] = upload_base_file['CY Weight'] * upload_base_file['Rate per KG'] * upload_base_file['Coefficient (Imported)']


In [111]:
upload_base_file['Recycling Fees (Total)'] = upload_base_file['Recycling Fees (Normal)'] + upload_base_file['Recycling Fees (Imported)']

In [112]:
upload_base_file = upload_base_file.drop(columns=['bst_copy', 'Material_y','bst_1', 'bst_y','bst'])

In [113]:
upload_base_file.columns

Index(['Item GTIN (EAN UPC Bar Code) ', 'Finished Product Part Name',
       'Finished Product Part Revision', 'Finished Product Part Title',
       'Sector', 'Sub Sector', 'Category', 'Brand', 'Primary Organization',
       'Secondary Organization', 'FPP Originator',
       'Number of COP per CUP (Items per Case)', 'Marketing Size',
       'Marketing Size UOM ', 'Statistical Unit Factor ', 'Base Unit Quantity',
       'Base Unit of Measure UOM ', 'Stat Unit Quantity', 'Consumer Items',
       'Region', 'MDO', 'Market of Sale', 'BOM Source', 'Reported Function',
       'BOM Packing Level', 'Quantity per CUP (Case)',
       'Quantity per CUP (Case) UOM', 'BOM Hierarchy (F/N)',
       'Packaging Under Product', 'Direct Parent Part Type',
       'Direct Parent Part Name', 'Direct Parent Part Revision',
       'Direct Parent Part Title', 'Packaging Part Type',
       'Packaging Part Name', 'Packaging Part Revision',
       'Packaging Part Title', 'Packaging Material Type',
       'Packagin

In [114]:
output_file_with_data = 'upload_base_file.xlsx'
upload_base_file.to_excel(output_file_with_data, index=False)

In [115]:
# material_avg = reporting_new.loc[reporting_new["Coefficient (Imported)"] != 0].groupby("Material")["Coefficient (Imported)"].mean().to_dict()

### Summery

In [116]:
Summery = upload_base_file

In [117]:
Summery = Summery[['Sub Sector','Category', 'CY Weight', 'Recycling Fees (Normal)','Final Classification','Final Material Classification','Final Material Classification_Copy','Source','Source_Copy', 'Business Classification']]

In [118]:
summary_1 = Summery.groupby(['Sub Sector','Category', 'Final Classification','Final Material Classification','Source','Business Classification','Final Material Classification_Copy','Source_Copy'])['Recycling Fees (Normal)'].sum().reset_index()

In [119]:
summary_1['Total Weight/Import'] = 'Total'

In [120]:
summary_1.rename(columns={'Recycling Fees (Normal)':'CY Cost'}, inplace=True)

In [121]:
summary_1

,Sub Sector,Category,Final Classification,Final Material Classification,Source,Business Classification,Final Material Classification_Copy,Source_Copy,CY Cost,Total Weight/Import
0,Hair Care,Hair Care,Glass Other,Container,JP-Sourced,"Cosmetics, Toothpaste, Other Toileries Manufac...",Container,JP-Sourced,5.755880e+04,Total
1,Hair Care,Hair Care,Glass Other,Container,JP-Sourced,"Cosmetics, Toothpaste, Other Toileries Manufac...",Container,NA,6.676196e+04,Total
2,Hair Care,Hair Care,NA,Container,Imported,"Cosmetics, Toothpaste, Other Toileries Manufac...",NA,Imported,0.000000e+00,Total
3,Hair Care,Hair Care,NA,Container,JP-Sourced,"Cosmetics, Toothpaste, Other Toileries Manufac...",NA,NA,0.000000e+00,Total
4,Hair Care,Hair Care,Paper,Container,Imported,"Cosmetics, Toothpaste, Other Toileries Manufac...",Container,Imported,2.441973e+04,Total
5,Hair Care,Hair Care,Paper,Container,JP-Sourced,"Cosmetics, Toothpaste, Other Toileries Manufac...",Container,JP-Sourced,1.498056e+03,Total
6,Hair Care,Hair Care,Paper,Container,JP-Sourced,"Cosmetics, Toothpaste, Other Toileries Manufac...",Container,NA,1.366027e+05,Total
7,Hair Care,Hair Care,Paper,Container,JP-Sourced,"Cosmetics, Toothpaste, Other Toileries Manufac...",NA,NA,9.108982e+00,Total
8,Hair Care,Hair Care,Plastic,Container,Imported,"Cosmetics, Toothpaste, Other Toileries Manufac...",Container,Imported,1.180920e+07,Total
9,Hair Care,Hair Care,Plastic,Container,Imported,"Cosmetics, Toothpaste, Other Toileries Manufac...",NA,Imported,5.163278e+04,Total


In [122]:
# summary_1 = pd.merge(summary_1, odi_cal[['Business Classification', 'Container/Wrapping']], left_on='Business Classification', right_on='Business Classification', how='left')

In [123]:
output_file_with_data = 'summary_1.xlsx'
summary_1.to_excel(output_file_with_data, index=False)

In [124]:
Summery_imp = upload_base_file

In [125]:
Summery_imp['Source'].unique()

array(['JP-Sourced', 'Imported'], dtype=object)

In [126]:
Summery_imp= Summery_imp[Summery_imp['Source'] == 'Imported']

In [127]:
Summery_imp = Summery_imp[['Sub Sector','Category', 'CY Weight', 'Recycling Fees (Imported)','Final Classification','Final Material Classification','Final Material Classification_Copy','Source','Source_Copy', 'Business Classification']]

In [128]:
Summery_imp = Summery_imp.groupby(['Sub Sector','Category', 'Final Classification','Final Material Classification','Source','Business Classification','Final Material Classification_Copy','Source_Copy'])['Recycling Fees (Imported)'].sum().reset_index()

In [129]:
Summery_imp['Total Weight/Import'] = 'Import'

In [130]:
Summery_imp.rename(columns={'Recycling Fees (Imported)':'CY Cost'}, inplace=True)

In [131]:
summery = pd.concat([summary_1, Summery_imp], ignore_index=True)

In [132]:
output_file_with_data = 'summery.xlsx'
summery.to_excel(output_file_with_data, index=False)

In [100]:
# pd.merge(reporting_1, unpivoted_df[['bst','Quantity','Material']],left_on='Concatenated Classification', right_on='bst', how='left')

In [101]:
# reporting_1

In [102]:
# reporting_1= reporting[reporting['Source'] != 'Imported']

# reporting_1=pd.merge(reporting_1, unpivoted_df[['bst','Quantity','Material']],left_on='Concatenated Classification', right_on='bst', how='inner')

In [103]:
# output_file_with_data = 'reporting_1.xlsx'
# reporting_1.to_excel(output_file_with_data, index=False)

In [104]:
# reporting_2= reporting[reporting['Source'] == 'Imported']
# reporting_2=pd.merge(reporting_2, unpivoted_df[['bst','Quantity','Material']],left_on='Concatenated Classification', right_on='bst', how='inner')

In [105]:
# output_file_with_data = 'reporting_2.xlsx'
# reporting_2.to_excel(output_file_with_data, index=False)

In [106]:
# Final_base_file = pd.concat([reporting_1, reporting_2], ignore_index=True)

In [58]:
# output_file_with_data = 'Final_base_file.xlsx'
# Final_base_file.to_excel(output_file_with_data, index=False)